In [1]:
from sklearn.datasets import fetch_openml
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
X, y = fetch_openml('mnist_784', return_X_y=True)

In [3]:
X.shape

(70000, 784)

In [4]:
pd.Series(y).value_counts()

1    7877
7    7293
3    7141
2    6990
9    6958
0    6903
6    6876
8    6825
4    6824
5    6313
dtype: int64

In [5]:
y

array(['5', '0', '4', ..., '4', '5', '6'], dtype=object)

In [6]:
X_scale = X/255.0

In [7]:
X_scale.max()

1.0

In [15]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X_scale,y,test_size=8000,random_state=10)

In [16]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

In [17]:
pca = PCA(.96)
train_pca = pca.fit_transform(xtrain)
test_pca = pca.transform(xtest)

In [18]:
train_pca.shape

(62000, 179)

In [19]:
log = LogisticRegression(C=10)
log.fit(train_pca,ytrain)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [20]:
print('train score:',log.score(train_pca,ytrain))

train score: 0.9200322580645162


In [21]:
from joblib import dump, load

In [22]:
dump(log, 'ProjectHandWritten.joblib') 

['ProjectHandWritten.joblib']

In [23]:
dump(pca,'HandWrittenpca.joblib')

['HandWrittenpca.joblib']

In [24]:
clf = load('ProjectHandWritten.joblib') 

In [25]:
clf.score(test_pca,ytest)

0.914375

In [26]:
pca1 = load('HandWrittenpca.joblib')

In [27]:
pca1.n_components_

179

In [ ]:
import tkinter as tk
from tkinter import *
from PIL import Image,ImageDraw
import matplotlib.pyplot as plt
import cv2
import numpy as np
global d
class ImageGenerator:
    def __init__(self,parent,posx,posy,*kwargs):
        self.parent = parent
        self.posx = posx
        self.posy = posy
        self.sizex = 400
        self.sizey = 300
        
        self.old_x=None
        self.old_y=None
        
       
        tk.Label(self.parent,text="Predicting Hand-Written Digits",fg="green",font=("",15,"bold")).place(x=50,y=10)
        
        self.drawing_area=tk.Canvas(self.parent,width=self.sizex,height=self.sizey)
        self.drawing_area.place(x=self.posx,y=self.posy)
        
        self.drawing_area.bind("<B1-Motion>", self.paint)
        self.drawing_area.bind("<ButtonRelease-1>", self.reset)
        
        self.button=tk.Button(self.parent,text="Save Image",width=10,bg='white',command=self.save)
        self.button.place(x=self.sizex/5,y=self.sizey+75)
        
        self.button1=tk.Button(self.parent,text="Clear",width=10,bg='white',command=self.clear)
        self.button1.place(x=(self.sizex/5)+80,y=self.sizey+75)
        
        self.button2=tk.Button(self.parent,text="Predict",width=10,bg='white',command=self.predict)
        self.button2.place(x=(self.sizex/5)+160,y=self.sizey+75)
        
       
        
        self.image=Image.new("RGB",(400,300),(255,255,255))
        self.draw=ImageDraw.Draw(self.image)

    def save(self):
        filename = "temp.jpg"
        self.image.save(filename)

    def clear(self):
        self.drawing_area.delete("all")
        self.image=Image.new("RGB",(400,300),(255,255,255))
        self.draw=ImageDraw.Draw(self.image)

    
    
    def paint(self, event):
        self.line_width = 10
        paint_color = 'black'
        if self.old_x and self.old_y:
            self.drawing_area.create_line(self.old_x, self.old_y, event.x, event.y,
                               width=self.line_width, fill=paint_color)
            self.draw.line(((self.old_x,self.old_y),(event.x,event.y)),(0,0,0),width=10)
        self.old_x = event.x
        self.old_y = event.y
        
    def reset(self, event):
        #self.c.delete("all")
        self.old_x, self.old_y = None, None
        
    
        
    def predict(self):      
        img =cv2.imread('temp.jpg')            
        gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray_img,(7,7),0)
        ret,th_img = cv2.threshold(blur, 110, 255, cv2.THRESH_BINARY_INV)
        ctrs, hier = cv2.findContours(th_img, cv2.RETR_EXTERNAL, cv2.THRESH_BINARY_INV)
        
        for contour in ctrs:
            [x,y,w,h] = cv2.boundingRect(contour)
            roi = gray_img[y-12:y+(h+12),x-12:x+(w+12)]
            roi = np.invert(roi)
            roi = cv2.resize(roi, (28, 28))
            arr = np.array(roi).astype(float)/255
            arr = arr.reshape(1,-1)
            test = pca.transform(arr)
            re=  clf.predict(test)
            cv2.putText(img, str(re[0]), (x-8,y-8),cv2.FONT_HERSHEY_DUPLEX, 2, (255, 0, 255), 3)
        
        cv2.imshow('final',img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
                        

if __name__ == "__main__":
    root=tk.Tk()
    root.wm_geometry("%dx%d+%d+%d" % (550, 450, 10, 10))
    root.config(bg='white')
    ImageGenerator(root,60,60)
    root.mainloop()
